# Data analysis books for UBike services
We require to install **pandas**, **metplotlib** and **seaborn** in the virtual environment. 


In [6]:
import pandas as pd
from matplotlib import pyplot as plt

pd.set_option('display.max_rows',30)
pd.set_option('large_repr', 'info')

# read data here
bikecsv = pd.read_csv('data/youbike_240421.csv') 

bikecsv.columns

Index(['_id', 'station_no', 'station_name', 'district', 'total', 'used',
       'empty', 'updated_time', 'info_time'],
      dtype='object')

## Before analyses, it requires us to know columns in csv data

**Here is the table show meaning or types of data. **

| Column Name | Type (Numerical/Category) | nullable | description |
| --- | --- | --- | --- |
| station_no | Categorical | non-null | Serial Number |  
| station_name | Categorical | non-null | Station name |  
| district | Categorical | non-null | stations in which districts |  
| total | Numerical |  non-null | total = used + empty| 
| used | Numerical |  non-null | available rental seats in stations | 
| empty | Numerical |  non-null | avaiable return seats in stations | 
|info_time | Date String | non-null | date time string in TZ GMT+8 |

Also, `info_time` can be categorize as workday or weekend to analyse 